In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import community as community_louvain  
import ast
import random

# Carica dati
edges = pd.read_csv('dataset/spoti/edges.csv')
nodes = pd.read_csv('dataset/spoti/nodes.csv')
nodes_unique = nodes.drop_duplicates(subset=['spotify_id'], keep='first')


In [ ]:
# ========== PARAMETRI DI SELEZIONE ==========
TOPLIST = 2000  # Numero di artisti da selezionare
SELECTION_CRITERION = "degree"  # Opzioni: "degree", "popularity", "betweenness", "closeness", "eigenvector", "random"
POPULARITY_FIELD = "popularity"  # Campo popolarità nel dataframe

# Parametri per la nazionalità
COUNTRY_FILTER = "uk"  # Opzioni: "italy", "france", "germany", "spain", "uk", None (per tutti)
COUNTRY_KEYWORDS = {
    "italy": ["ital"],
    "france": ["fran"],
    "germany": ["german"],
    "spain": ["spanish"],
    "uk": ["british", "uk"],
    "nordic": ["swedish", "norwegian", "danish", "finnish"],
}

SEED = 42  # Per riproducibilità nella selezione random
np.random.seed(SEED)
random.seed(SEED)

In [ ]:
def is_from_country(genres, country_filter=None):
    """
    Verifica se un artista appartiene a un paese specifico basandosi sui generi.
    
    Parametri:
    - genres: stringa o lista di generi
    - country_filter: chiave del paese (es. "italy", "france") o None per nessun filtro
    
    Returns:
    - Boolean
    """
    if country_filter is None:
        return True
    
    if country_filter not in COUNTRY_KEYWORDS:
        print(f"⚠️ Paese '{country_filter}' non riconosciuto. Usando tutti gli artisti.")
        return True
    
    keywords = COUNTRY_KEYWORDS[country_filter]
    
    if isinstance(genres, str):
        genres_lower = genres.lower()
        return any(keyword in genres_lower for keyword in keywords)
    
    return False

In [ ]:
def parse_genres(genres_field):
    """Converte stringa rappresentante lista in lista Python"""
    if pd.isna(genres_field):
        return []
    if isinstance(genres_field, str):
        try:
            parsed = ast.literal_eval(genres_field)
            return parsed if isinstance(parsed, list) else []
        except:
            return []
    if isinstance(genres_field, list):
        return genres_field
    return []



In [ ]:
# ========== FUNZIONE DI SELEZIONE TOP NODI ==========
def select_top_nodes(graph, nodes_df, n=100, criterion="degree", popularity_field="popularity"):
    """
    Seleziona i top n nodi secondo diversi criteri di centralità.
    
    Parametri:
    - graph: grafo NetworkX
    - nodes_df: dataframe con attributi degli artisti
    - n: numero di nodi da selezionare
    - criterion: metrica di selezione ("degree", "popularity", "betweenness", "closeness", "eigenvector", "random")
    - popularity_field: nome colonna popolarità
    
    Returns:
    - Lista di nodi selezionati
    """
    
    if criterion == "degree":
        # Centralità di grado: numero di collaborazioni
        metric = dict(graph.degree())
        description = "numero di collaborazioni"
        
    elif criterion == "popularity":
        # Popolarità su Spotify
        metric = {}
        for node in graph.nodes():
            if node in nodes_df['spotify_id'].values:
                pop = nodes_df.loc[nodes_df['spotify_id'] == node, popularity_field].values
                metric[node] = pop[0] if len(pop) > 0 else 0
            else:
                metric[node] = 0
        description = "popolarità Spotify"
        
    elif criterion == "betweenness":
        # Centralità di intermediazione: broker tra comunità
        metric = nx.betweenness_centrality(graph)
        description = "centralità di intermediazione"
        
    elif criterion == "closeness":
        # Centralità di vicinanza: distanza media dagli altri
        metric = nx.closeness_centrality(graph)
        description = "centralità di vicinanza"
        
    elif criterion == "eigenvector":
        # Centralità di autovettore: connessione a nodi importanti
        try:
            metric = nx.eigenvector_centrality(graph, max_iter=1000)
            description = "centralità di autovettore"
        except:
            print("⚠️ Eigenvector centrality non convergente, uso degree")
            metric = dict(graph.degree())
            description = "grado (fallback)"
            
    elif criterion == "random":
        # Selezione casuale
        all_nodes = list(graph.nodes())
        random.shuffle(all_nodes)
        top_nodes = all_nodes[:n]
        
        print(f"\n[SELEZIONE TOP {n}]")
        print(f"Criterio: {criterion} (seed={SEED})")
        print(f"Primi 5 artisti selezionati casualmente:")
        for i, node in enumerate(top_nodes[:5], 1):
            name = graph.nodes[node].get('name', node)
            print(f"  {i}. {name}")
        
        return top_nodes
    else:
        raise ValueError(f"Criterio '{criterion}' non valido. Opzioni: degree, popularity, betweenness, closeness, eigenvector, random")
    
    # Ordina e seleziona top n (non per random, già gestito sopra)
    sorted_nodes = sorted(metric.items(), key=lambda x: x[1], reverse=True)
    top_nodes = [n for n, v in sorted_nodes[:n]]
    
    print(f"\n[SELEZIONE TOP {n}]")
    print(f"Criterio: {criterion} ({description})")
    print(f"Top 5 artisti selezionati:")
    for i, (node, value) in enumerate(sorted_nodes[:5], 1):
        name = graph.nodes[node].get('name', node)
        print(f"  {i}. {name} (valore: {value:.3f})")
    
    return top_nodes

In [ ]:
# Filtra artisti per paese
nodes_country = nodes_unique[
    nodes_unique.apply(lambda row: is_from_country(row['genres'], COUNTRY_FILTER), axis=1)
]

# Converti generi da stringa a lista
nodes_country['genres'] = nodes_country['genres'].apply(parse_genres)
print(f"✓ Generi convertiti: {sum(nodes_country['genres'].apply(len) > 0)} artisti con genere")

# Estrai IDs degli artisti del paese selezionato
country_ids = set(nodes_country['spotify_id'])
print(f"Artisti {'italiani' if COUNTRY_FILTER == 'italy' else COUNTRY_FILTER if COUNTRY_FILTER else 'globali'} identificati: {len(country_ids)}")

# Filtra solo collaborazioni tra artisti dello stesso paese
tt = edges[edges['id_0'].isin(country_ids) & edges['id_1'].isin(country_ids)]

# Creazione grafo completo
G_country = nx.Graph()
G_country.add_edges_from(tt[['id_0', 'id_1']].values)

# Aggiungi attributi dei nodi
attr_dict = nodes_country.set_index('spotify_id').to_dict('index')
nx.set_node_attributes(G_country, attr_dict)

print(f"\n[GRAFO COMPLETO - {COUNTRY_FILTER.upper() if COUNTRY_FILTER else 'GLOBALE'}]")
print(f"Nodi (artisti): {G_country.number_of_nodes()}")
print(f"Archi (collaborazioni): {G_country.number_of_edges()}")

In [ ]:
# ========== CREAZIONE SOTTOGRAFO ANALISI ==========
# Seleziona i top artisti
top_nodes = select_top_nodes(G_country, nodes_country, n=TOPLIST, criterion=SELECTION_CRITERION)

# Crea sottografo con solo i top artisti e le loro collaborazioni
G_top = G_country.subgraph(top_nodes).copy()

print(f"\n[GRAFO TOP {TOPLIST}]")
print(f"Nodi: {G_top.number_of_nodes()}")
print(f"Archi: {G_top.number_of_edges()}")

# ========== STATISTICHE GENERALIZZATE ==========
print(f"\n[SUMMARY PARAMETRI]")
print(f"  Paese: {COUNTRY_FILTER.upper() if COUNTRY_FILTER else 'GLOBALE'}")
print(f"  Criterio selezione: {SELECTION_CRITERION}")
print(f"  Top: {TOPLIST} artisti")
print(f"  Grafo completo: {G_country.number_of_nodes()} nodi, {G_country.number_of_edges()} archi")
print(f"  Grafo analisi: {G_top.number_of_nodes()} nodi, {G_top.number_of_edges()} archi")

In [ ]:
print("\n" + "="*60)
print("1. PROPRIETÀ GLOBALI DELLA RETE")
print("="*60)

# Densità: rapporto tra archi esistenti e possibili
density = nx.density(G_top)
print(f"\nDensità: {density:.4f}")
print(f"  → La rete è {'molto' if density > 0.1 else 'poco'} connessa")

# Componenti connesse
components = list(nx.connected_components(G_top))
print(f"\nComponenti connesse: {len(components)}")
print(f"  → Dimensione componente principale: {len(max(components, key=len))} nodi")

# Lavora sulla componente gigante
G_main = G_top.subgraph(max(components, key=len)).copy()

# Diametro e cammino medio (solo se connesso)
if nx.is_connected(G_main):
    diameter = nx.diameter(G_main)
    avg_path = nx.average_shortest_path_length(G_main)
    print(f"\nDiametro: {diameter}")
    print(f"  → Massima distanza tra due artisti: {diameter} passaggi")
    print(f"\nCammino medio: {avg_path:.3f}")
    print(f"  → Distanza media tra artisti: {avg_path:.3f} collaborazioni")
else:
    print("\nGrafo non completamente connesso, diametro non calcolabile")

# Coefficiente di clustering
clustering_coeff = nx.average_clustering(G_main)
print(f"\nCoefficient di clustering medio: {clustering_coeff:.4f}")
print(f"  → I collaboratori di un artista {'spesso' if clustering_coeff > 0.3 else 'raramente'} collaborano tra loro")


In [ ]:
print("\n" + "="*60)
print("2. DISTRIBUZIONE DEI GRADI")
print("="*60)

# Calcola gradi
degrees = [G_main.degree(n) for n in G_main.nodes()]
degree_count = Counter(degrees)

# Statistiche
avg_degree = np.mean(degrees)
median_degree = np.median(degrees)
max_degree = max(degrees)

print(f"\nGrado medio: {avg_degree:.2f}")
print(f"Grado mediano: {median_degree:.0f}")
print(f"Grado massimo: {max_degree}")

# Trova artisti più connessi
top_degree = sorted(G_main.degree(), key=lambda x: x[1], reverse=True)[:10]
print(f"\nTop 10 artisti per numero di collaborazioni:")
for i, (node, deg) in enumerate(top_degree, 1):
    name = G_main.nodes[node].get('name', node)
    print(f"  {i}. {name}: {deg} collaborazioni")

# Test distribuzione power-law
print(f"\nLa distribuzione {'segue' if max_degree > 5*avg_degree else 'non segue'} un pattern scale-free")
print(f"  → {'Pochi hub dominano' if max_degree > 5*avg_degree else 'Rete più omogenea'}")


In [ ]:
print("\n" + "="*60)
print("3. MISURE DI CENTRALITÀ")
print("="*60)

# Calcola diverse centralità
degree_cent = nx.degree_centrality(G_main)
betweenness_cent = nx.betweenness_centrality(G_main)
closeness_cent = nx.closeness_centrality(G_main)

try:
    eigenvector_cent = nx.eigenvector_centrality(G_main, max_iter=1000)
except:
    eigenvector_cent = {n: 0 for n in G_main.nodes()}

# Crea dataframe comparativo
centrality_df = pd.DataFrame({
    'artist': [G_main.nodes[n].get('name', n) for n in G_main.nodes()],
    'degree': [degree_cent[n] for n in G_main.nodes()],
    'betweenness': [betweenness_cent[n] for n in G_main.nodes()],
    'closeness': [closeness_cent[n] for n in G_main.nodes()],
    'eigenvector': [eigenvector_cent[n] for n in G_main.nodes()]
})

print("\n[DEGREE CENTRALITY] - Artisti più connessi:")
top_deg = centrality_df.nlargest(5, 'degree')
for idx, row in top_deg.iterrows():
    print(f"  {row['artist']}: {row['degree']:.4f}")

print("\n[BETWEENNESS CENTRALITY] - Artisti bridge tra comunità:")
top_bet = centrality_df.nlargest(5, 'betweenness')
for idx, row in top_bet.iterrows():
    print(f"  {row['artist']}: {row['betweenness']:.4f}")

print("\n[CLOSENESS CENTRALITY] - Artisti centrali nella rete:")
top_clo = centrality_df.nlargest(5, 'closeness')
for idx, row in top_clo.iterrows():
    print(f"  {row['artist']}: {row['closeness']:.4f}")

print("\n[EIGENVECTOR CENTRALITY] - Artisti connessi ad altri importanti:")
top_eig = centrality_df.nlargest(5, 'eigenvector')
for idx, row in top_eig.iterrows():
    print(f"  {row['artist']}: {row['eigenvector']:.4f}")


In [ ]:
print("\n" + "="*60)
print("4. RILEVAMENTO COMUNITÀ")
print("="*60)

# Algoritmo Louvain per community detection
partition = community_louvain.best_partition(G_main)

# Aggiungi community come attributo
nx.set_node_attributes(G_main, partition, 'community')

# Analisi comunità
num_communities = len(set(partition.values()))
print(f"\nNumero di comunità rilevate: {num_communities}")

# Dimensione comunità
community_sizes = Counter(partition.values())
print(f"\nDistribuzione dimensioni comunità:")
for comm_id, size in sorted(community_sizes.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(f"  Comunità {comm_id}: {size} artisti")

# Modularity
modularity = community_louvain.modularity(partition, G_main)
print(f"\nModularità: {modularity:.4f}")
print(f"  → {'Forte' if modularity > 0.4 else 'Moderata' if modularity > 0.3 else 'Debole'} struttura a comunità")

# Artisti per comunità (top 3 comunità più grandi)
print(f"\nArtisti principali per comunità (top 3 comunità):")
for comm_id, size in sorted(community_sizes.items(), key=lambda x: x[1], reverse=True)[:3]:
    print(f"\n  Comunità {comm_id} ({size} artisti):")
    comm_nodes = [n for n, c in partition.items() if c == comm_id]
    comm_degrees = [(n, G_main.degree(n)) for n in comm_nodes]
    top_in_comm = sorted(comm_degrees, key=lambda x: x[1], reverse=True)[:5]
    for node, deg in top_in_comm:
        name = G_main.nodes[node].get('name', node)
        print(f"    - {name} ({deg} collab.)")


In [ ]:
print("\n" + "="*60)
print("5. BRIDGE ANALYSIS - CONNETTORI TRA COMUNITÀ")
print("="*60)

# Identifica edge betweenness
edge_betweenness = nx.edge_betweenness_centrality(G_main)

# Top bridge edges
top_bridges = sorted(edge_betweenness.items(), key=lambda x: x[1], reverse=True)[:10]

print("\nTop 10 collaborazioni-bridge:")
for i, ((u, v), score) in enumerate(top_bridges, 1):
    name_u = G_main.nodes[u].get('name', u)
    name_v = G_main.nodes[v].get('name', v)
    comm_u = partition[u]
    comm_v = partition[v]
    is_bridge = "✓" if comm_u != comm_v else "✗"
    print(f"  {i}. {name_u} ↔ {name_v} (score: {score:.4f}) [Bridge: {is_bridge}]")

# Constraint (Burt's structural holes)
constraint = nx.constraint(G_main)
low_constraint = sorted(constraint.items(), key=lambda x: x[1])[:10]

print("\nArtisti con accesso a structural holes (basso constraint):")
for i, (node, const) in enumerate(low_constraint, 1):
    name = G_main.nodes[node].get('name', node)
    print(f"  {i}. {name} (constraint: {const:.4f})")


In [ ]:
print("\n" + "="*60)
print("6. CLIQUES E GRUPPI COESI")
print("="*60)

# Trova cliques massimali
cliques = list(nx.find_cliques(G_main))
clique_sizes = [len(c) for c in cliques]

print(f"\nNumero totale di cliques: {len(cliques)}")
print(f"Dimensione massima clique: {max(clique_sizes)}")
print(f"Dimensione media clique: {np.mean(clique_sizes):.2f}")

# Top 5 cliques più grandi
largest_cliques = sorted(cliques, key=len, reverse=True)[:5]
print(f"\nTop 5 cliques più grandi:")
for i, clique in enumerate(largest_cliques, 1):
    print(f"\n  Clique {i} ({len(clique)} artisti):")
    for node in clique[:10]:  # Mostra max 10
        name = G_main.nodes[node].get('name', node)
        print(f"    - {name}")

# K-core decomposition
k_cores = nx.core_number(G_main)
max_k = max(k_cores.values())

print(f"\nK-core massimo: {max_k}")
print(f"  → Esiste un nucleo di artisti con almeno {max_k} connessioni reciproche")

# Artisti nel k-core massimo
max_core_nodes = [n for n, k in k_cores.items() if k == max_k]
print(f"\nArtisti nel {max_k}-core ({len(max_core_nodes)} artisti):")
for node in sorted(max_core_nodes, key=lambda n: G_main.degree(n), reverse=True)[:10]:
    name = G_main.nodes[node].get('name', node)
    degree = G_main.degree(node)
    print(f"  - {name} ({degree} collab.)")


In [ ]:
print("\n" + "="*60)
print("7. PROPRIETÀ SMALL-WORLD")
print("="*60)

# Usa il sottografo principale della tua selezione
# (rinomina G_main → G_top o usa qualunque variabile rappresenti il grafo in analisi)
G_analysis = G_top  # Puoi facilmente cambiarlo in G_country o altro in base ai tuoi settings

# Calcola coefficienti su G_analysis
C_real = nx.average_clustering(G_analysis)
if nx.is_connected(G_analysis):
    L_real = nx.average_shortest_path_length(G_analysis)
else:
    L_real = None

# Parametri base del grafo per generare confronto random
n = G_analysis.number_of_nodes()
m = G_analysis.number_of_edges()
if n > 1:
    p = 2 * m / (n * (n - 1))
else:
    p = 0

# Genera grafo random Erdős–Rényi con stesso numero di nodi e archi attesi
G_random = nx.erdos_renyi_graph(n, p, seed=SEED)
C_random = nx.average_clustering(G_random)

if nx.is_connected(G_random):
    L_random = nx.average_shortest_path_length(G_random)
else:
    # Se non connesso, usa componente gigante
    largest_cc = max(nx.connected_components(G_random), key=len)
    G_random_main = G_random.subgraph(largest_cc)
    L_random = nx.average_shortest_path_length(G_random_main)

# Output generalizzato
print(f"\nRete musicale selezionata:")
print(f"  Clustering: {C_real:.4f}")
if L_real:
    print(f"  Cammino medio: {L_real:.4f}")

print(f"\nGrafo random equivalente:")
print(f"  Clustering: {C_random:.4f}")
print(f"  Cammino medio: {L_random:.4f}")

# Test small-world
if L_real:
    sigma = (C_real / C_random) / (L_real / L_random)
    print(f"\nSmall-world coefficient (σ): {sigma:.4f}")
    if sigma > 1:
        print(f"  ✓ La rete selezionata ha proprietà SMALL-WORLD")
        print(f"    → Alto clustering locale + brevi distanze globali")
    else:
        print(f"  ✗ La rete selezionata non mostra forti proprietà small-world")
else:
    print("\nCammino medio non calcolabile (rete non connessa)")


In [ ]:
print("\n" + "="*60)
print("8. ANALISI PER GENERE MUSICALE")
print("="*60)

# Verifica se esiste campo genere
if 'genres' in G_main.nodes[list(G_main.nodes())[0]]:
    
    # Estrai generi principali
    genre_dict = {}
    for node in G_main.nodes():
        genres = G_main.nodes[node].get('genres', [])
        if isinstance(genres, list) and len(genres) > 0:
            # Prendi primo genere
            genre_dict[node] = genres[0]
        else:
            genre_dict[node] = 'Unknown'
    
    # Distribuzione generi
    genre_count = Counter(genre_dict.values())
    print(f"\nDistribuzione generi (top 10):")
    for genre, count in genre_count.most_common(10):
        print(f"  {genre}: {count} artisti")
    
    # Assortatività per genere
    nx.set_node_attributes(G_main, genre_dict, 'genre')
    
    # Collaborazioni intra vs inter-genere
    intra_genre = 0
    inter_genre = 0
    for u, v in G_main.edges():
        if genre_dict[u] == genre_dict[v]:
            intra_genre += 1
        else:
            inter_genre += 1
    
    total_edges = intra_genre + inter_genre
    print(f"\nCollaborazioni intra-genere: {intra_genre} ({100*intra_genre/total_edges:.1f}%)")
    print(f"Collaborazioni inter-genere: {inter_genre} ({100*inter_genre/total_edges:.1f}%)")
    
    if intra_genre > inter_genre:
        print("  → Gli artisti tendono a collaborare dentro lo stesso genere")
    else:
        print("  → Forte cross-pollination tra generi diversi")
    
else:
    print("\nAttributo 'genres' non disponibile nel dataset")


In [ ]:
import os

# --- Prepara la rete per export ---
# Converti tipi non supportati (list, dict, tuple) in stringa:
for n, data in G_top.nodes(data=True):
    for k, v in data.items():
        if isinstance(v, (list, dict, tuple)):
            G_top.nodes[n][k] = str(v)

# Prepara nome directory e file:
country = COUNTRY_FILTER if COUNTRY_FILTER else "all"
crit = SELECTION_CRITERION
topn = TOPLIST
export_dir = "exports_gexf"
os.makedirs(export_dir, exist_ok=True)
filename = f"{country}_{crit}_top{topn}.gexf"
filepath = os.path.join(export_dir, filename)

# Exporta il grafo
nx.write_gexf(G_top, filepath)

print(f"\n✓ Esportazione completata!")
print(f"  File GEXF salvato in: {filepath}")
print(f"  → Aprilo in Gephi per analisi/visualizzazione.")
